First complete the following tasks

- Change time format of both files to Pandas datetime

- Join the two data frames according to time

- Calculate the hourly bill paid (using information about the price and the consumption)

- Calculated grouped values of daily, weekly or monthly consumption, bill, average price and average temperature

In [276]:

import pandas as pd

electricity_data = pd.read_csv('Electricity_20-09-2024.csv',delimiter= ';')
price_data = pd.read_csv('sahkon-hinta-010121-240924.csv')
display(electricity_data.head())
display(price_data.head())

,Time,Energy (kWh),Energy night(kWh),Energy day (kWh),Temperature
0,1.1.2020 0:00,"5,210","5,210",NaN,"-3,4"
1,1.1.2020 1:00,"4,774","4,774",NaN,"-2,5"
2,1.1.2020 2:00,"4,481","4,481",NaN,"-4,1"
3,1.1.2020 3:00,"4,110","4,110",NaN,"-2,7"
4,1.1.2020 4:00,"1,531","1,531",NaN,"-4,5"


,Time,Price (cent/kWh)
0,01-01-2021 00:00:00,3.094
1,01-01-2021 01:00:00,3.019
2,01-01-2021 02:00:00,2.974
3,01-01-2021 03:00:00,2.941
4,01-01-2021 04:00:00,2.943


In [277]:
#check datatype
print('electricity_datatype',type(electricity_data['Time'][3]))
print('price_datatype',type(price_data['Time'][3]))

electricity_datatype <class 'str'>
price_datatype <class 'str'>


In [278]:
#Change time format of both files to Pandas datetime


electricity_data['Time'] = pd.to_datetime(electricity_data['Time'], format = '%d.%m.%Y %H:%M',errors='coerce')
price_data['Time'] = pd.to_datetime(price_data['Time'], format = '%d-%m-%Y %H:%M:%S')

display(electricity_data.head())
display(price_data.head())

,Time,Energy (kWh),Energy night(kWh),Energy day (kWh),Temperature
0,2020-01-01 00:00:00,"5,210","5,210",NaN,"-3,4"
1,2020-01-01 01:00:00,"4,774","4,774",NaN,"-2,5"
2,2020-01-01 02:00:00,"4,481","4,481",NaN,"-4,1"
3,2020-01-01 03:00:00,"4,110","4,110",NaN,"-2,7"
4,2020-01-01 04:00:00,"1,531","1,531",NaN,"-4,5"


,Time,Price (cent/kWh)
0,2021-01-01 00:00:00,3.094
1,2021-01-01 01:00:00,3.019
2,2021-01-01 02:00:00,2.974
3,2021-01-01 03:00:00,2.941
4,2021-01-01 04:00:00,2.943


In [279]:
#Join the two data frames according to time
df = pd.merge(electricity_data, price_data, on = 'Time',how='inner')
df.head()


,Time,Energy (kWh),Energy night(kWh),Energy day (kWh),Temperature,Price (cent/kWh)
0,2021-01-01 00:00:00,"1,988","1,988",NaN,"-0,9",3.094
1,2021-01-01 01:00:00,"1,462","1,462",NaN,"-1,3",3.019
2,2021-01-01 02:00:00,"1,101","1,101",NaN,"-1,4",2.974
3,2021-01-01 03:00:00,"1,031","1,031",NaN,"-1,9",2.941
4,2021-01-01 04:00:00,"1,044","1,044",NaN,"-1,9",2.943


In [280]:
# Calculate the hourly bill paid (using information about the price and the consumption)


#Replace commas with dots and convert to numeric in the electricity data
electricity_data['Energy (kWh)'] = electricity_data['Energy (kWh)'].str.replace(',', '.').astype(float)

#Merge the datasets on the 'Time' column
df = pd.merge(electricity_data, price_data, on = 'Time')

#Calculate the hourly bill by multiplying consumption (kWh) by the price (cent/kWh) converted to euros
df['Hourly Bill (€)'] = df['Energy (kWh)'] * (df['Price (cent/kWh)'] / 100)

display(df[['Time', 'Energy (kWh)', 'Price (cent/kWh)', 'Hourly Bill (€)']].head())



,Time,Energy (kWh),Price (cent/kWh),Hourly Bill (€)
0,2021-01-01 00:00:00,1.988,3.094,0.061509
1,2021-01-01 01:00:00,1.462,3.019,0.044138
2,2021-01-01 02:00:00,1.101,2.974,0.032744
3,2021-01-01 03:00:00,1.031,2.941,0.030322
4,2021-01-01 04:00:00,1.044,2.943,0.030725


In [281]:
# Calculated grouped values of daily, weekly or monthly consumption, bill, average price and average temperature

# Ensure the relevant columns are numeric, handling any conversion issues
df['Energy (kWh)'] = pd.to_numeric(df['Energy (kWh)'], errors='coerce')
df['Price (cent/kWh)'] = pd.to_numeric(df['Price (cent/kWh)'], errors='coerce')
df['Hourly Bill (€)'] = pd.to_numeric(df['Hourly Bill (€)'], errors='coerce')
df['Temperature'] = pd.to_numeric(df['Temperature'], errors='coerce')

# Group by daily, weekly, and monthly 
df_group_daily = df.groupby(pd.Grouper(key = 'Time', freq = 'd',)).agg({'Energy (kWh)' :'sum', 'Price (cent/kWh)': 'mean', 'Temperature': 'mean'}).reset_index()
df_group_weekly = df.groupby(pd.Grouper(key = 'Time', freq = 'W')).agg({'Energy (kWh)' :'sum', 'Price (cent/kWh)': 'mean', 'Temperature':'mean'}).reset_index()
df_group_monthly = df.groupby(pd.Grouper(key = 'Time', freq = 'ME')).agg({'Energy (kWh)' :'sum', 'Price (cent/kWh)': 'mean', 'Temperature':'mean'}).reset_index()

print('Daily Data:\n', df_group_daily.head())
print('\nWeekly Data:\n', df_group_weekly.head())
print('\nMonthly Data:\n', df_group_monthly.head())

Daily Data:
         Time  Energy (kWh)  Price (cent/kWh)  Temperature
0 2021-01-01        36.701          3.255167          NaN
1 2021-01-02        43.606          4.068750          NaN
2 2021-01-03        59.927          3.202458          NaN
3 2021-01-04        77.953          5.376833          NaN
4 2021-01-05        74.834          5.863458          NaN

Weekly Data:
         Time  Energy (kWh)  Price (cent/kWh)  Temperature
0 2021-01-03       140.234          3.508792          NaN
1 2021-01-10       424.256          6.849715          NaN
2 2021-01-17         0.000               NaN          NaN
3 2021-01-24         0.000               NaN          NaN
4 2021-01-31         0.000               NaN          NaN

Monthly Data:
         Time  Energy (kWh)  Price (cent/kWh)  Temperature
0 2021-01-31       564.490          5.736074          NaN
1 2021-02-28       678.368          7.808792          NaN
2 2021-03-31       504.998          5.158486          NaN
3 2021-04-30       318.316  